In [71]:
import os
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("stream").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [39]:
covid = spark.read.csv('covid.csv', sep=';',inferSchema=True, header=True)

In [40]:
for_sql = spark.read.option('header',True).csv('covid.csv').createOrReplaceTempView('covid2')

In [41]:
covid

DataFrame[iso_code: string, continent: string, location: string, date: string, total_cases: double, new_cases: double, new_cases_smoothed: double, total_deaths: double, new_deaths: double, new_deaths_smoothed: double, total_cases_per_million: double, new_cases_per_million: string, new_cases_smoothed_per_million: string, total_deaths_per_million: string, new_deaths_per_million: string, new_deaths_smoothed_per_million: string, reproduction_rate: string, icu_patients: double, icu_patients_per_million: string, hosp_patients: double, hosp_patients_per_million: string, weekly_icu_admissions: double, weekly_icu_admissions_per_million: string, weekly_hosp_admissions: double, weekly_hosp_admissions_per_million: string, total_tests: double, new_tests: double, total_tests_per_thousand: string, new_tests_per_thousand: string, new_tests_smoothed: double, new_tests_smoothed_per_thousand: string, positive_rate: double, tests_per_case: string, tests_units: string, total_vaccinations: double, people_va

In [6]:
covid.select('location').show()

+-----------+
|   location|
+-----------+
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
+-----------+
only showing top 20 rows



#### Group by continent

In [7]:
continents = covid.groupBy('continent').count()
continents.show()

+-------------+-----+
|    continent|count|
+-------------+-----+
|       Europe|11770|
|       Africa|11770|
|         null| 2782|
|North America| 7918|
|South America| 2782|
|      Oceania| 4759|
|         Asia|10914|
+-------------+-----+



#### Extracting data for Europe

In [8]:
europe = covid.filter(col('continent') == 'Europe')
europe.count()

11770

#### Extracting data for Asia

In [96]:
asia = covid.filter(col('continent') == 'Asia')
asia.count()

10914

#### Show avg number of new cases from June 2021 - December 2021 in Europe

In [97]:
europe.select(avg('new_cases')).show()

+------------------+
|    avg(new_cases)|
+------------------+
|3943.2531247018414|
+------------------+



In [98]:
t_cases_europe = europe.groupby('location').agg(sum('total_cases').alias('total_cases_sum'))
t_cases_europe.show()

+--------------------+---------------+
|            location|total_cases_sum|
+--------------------+---------------+
|             Belgium|   2.85802476E8|
|             Albania|    3.4800192E7|
|             Belarus|    1.1286723E8|
|             Croatia|    9.5219245E7|
|             Andorra|      3359930.0|
|             Czechia|   3.88283214E8|
|             Denmark|    8.2310132E7|
|              Cyprus|    2.4118094E7|
|Bosnia and Herzeg...|    5.0000976E7|
|            Bulgaria|   1.12641046E8|
|             Austria|   1.72615342E8|
|              Jersey|           null|
|             Germany|   9.62817071E8|
|              France|   1.47750814E9|
|              Greece|    1.4084059E8|
|              Kosovo|     2.983407E7|
|             Finland|    3.0297431E7|
|            Guernsey|           null|
|         Isle of Man|      1498997.0|
|               Italy|   9.96233156E8|
+--------------------+---------------+
only showing top 20 rows



In [99]:
sorted_t_cases_europe = t_cases.select('location','total_cases_sum').orderBy('total_cases_sum', ascending = False)
sorted_t_cases_europe.show()

+--------------+---------------+
|      location|total_cases_sum|
+--------------+---------------+
|United Kingdom|  1.619123749E9|
|        Russia|  1.573120097E9|
|        France|   1.47750814E9|
|         Spain|  1.010464279E9|
|         Italy|   9.96233156E8|
|       Germany|   9.62817071E8|
|        Poland|   6.61171393E8|
|       Ukraine|   5.90001419E8|
|   Netherlands|   4.50591578E8|
|       Czechia|   3.88283214E8|
|       Romania|   2.86600834E8|
|       Belgium|   2.85802476E8|
|        Sweden|    2.4546081E8|
|      Portugal|   2.22233108E8|
|        Serbia|   2.00326746E8|
|      Slovakia|   1.92099349E8|
|       Hungary|   1.91250969E8|
|   Switzerland|   1.80929305E8|
|       Austria|   1.72615342E8|
|        Greece|    1.4084059E8|
+--------------+---------------+
only showing top 20 rows



#### Show avg number of new cases from June 2021 - December 2021 in Asia

In [100]:
asia.select(avg('new_cases')).show()

+------------------+
|    avg(new_cases)|
+------------------+
|3284.9509059029806|
+------------------+



In [101]:
t_cases_asia = asia.groupby('location').agg(sum('total_cases').alias('total_cases_sum'))
t_cases_asia.show()

+-----------+---------------+
|   location|total_cases_sum|
+-----------+---------------+
|Afghanistan|    3.0831741E7|
|   Cambodia|    1.9860492E7|
|      China|    2.2810823E7|
| Bangladesh|   2.93199187E8|
|     Bhutan|       528293.0|
| Azerbaijan|    9.7340286E7|
|    Armenia|    5.7813011E7|
|     Brunei|      1417798.0|
|    Bahrain|    5.8182997E7|
|       Iraq|   3.87627555E8|
|     Jordan|   1.79304519E8|
|      India|  7.016140517E9|
|     Kuwait|    8.4716886E7|
|       Iran|   1.05013922E9|
|       Laos|      6646227.0|
|  Hong Kong|      2605801.0|
|     Israel|   2.40009265E8|
|    Georgia|   1.27268874E8|
|  Indonesia|   7.79570564E8|
| Kyrgyzstan|    3.5294482E7|
+-----------+---------------+
only showing top 20 rows



In [103]:
sorted_t_cases_asia = t_cases_asia.select('location','total_cases_sum').orderBy('total_cases_sum', ascending = False)
sorted_t_cases_asia.show()

+--------------------+---------------+
|            location|total_cases_sum|
+--------------------+---------------+
|               India|  7.016140517E9|
|              Turkey|  1.495669191E9|
|                Iran|   1.05013922E9|
|           Indonesia|   7.79570564E8|
|         Philippines|   4.65841792E8|
|                Iraq|   3.87627555E8|
|            Malaysia|   3.87450342E8|
|               Japan|   2.94283706E8|
|          Bangladesh|   2.93199187E8|
|            Thailand|   2.72882453E8|
|            Pakistan|   2.47161525E8|
|              Israel|   2.40009265E8|
|              Jordan|   1.79304519E8|
|          Kazakhstan|   1.77236573E8|
|               Nepal|   1.60128081E8|
|United Arab Emirates|   1.50382024E8|
|             Vietnam|   1.35128123E8|
|             Lebanon|   1.30758781E8|
|             Georgia|   1.27268874E8|
|        Saudi Arabia|   1.13359431E8|
+--------------------+---------------+
only showing top 20 rows



#### Correlation between

In [74]:
data = covid.select('location','new_vaccinations','new_cases').filter((col('location') == 'Poland'))

In [90]:
data2 = covid.select('location','new_vaccinations','new_cases').filter((col('location') == 'Germany'))

In [91]:
data.show()

+--------+----------------+---------+
|location|new_vaccinations|new_cases|
+--------+----------------+---------+
|  Poland|        451423.0|    585.0|
|  Poland|        638504.0|    659.0|
|  Poland|        152450.0|    565.0|
|  Poland|        426942.0|    319.0|
|  Poland|        271933.0|    413.0|
|  Poland|        108491.0|    312.0|
|  Poland|        206262.0|    192.0|
|  Poland|        396464.0|    401.0|
|  Poland|        503054.0|    560.0|
|  Poland|        388308.0|    378.0|
|  Poland|        417376.0|    340.0|
|  Poland|        314955.0|    236.0|
|  Poland|        109520.0|    226.0|
|  Poland|        217328.0|    139.0|
|  Poland|            null|    211.0|
|  Poland|            null|    242.0|
|  Poland|        401483.0|    215.0|
|  Poland|        322386.0|    190.0|
|  Poland|        215615.0|    168.0|
|  Poland|         72093.0|    133.0|
+--------+----------------+---------+
only showing top 20 rows



In [92]:
corr = data.stat.corr('new_vaccinations','new_cases')

In [93]:
corr

-0.07256321751110345

In [94]:
corr2 = data2.stat.corr('new_vaccinations','new_cases')

In [95]:
corr2

0.284041836339982

For Poland, the correlation coefficient of -0.072 indicates a weak negative relationship between 'new_vaccinations' and 'new_cases'. This means that in Poland, there is a slight tendency for the number of new cases to decrease slightly as the number of new vaccinations increases. However, this relationship is very weak and almost negligible.

For Germany, the correlation coefficient of 0.28 indicates a moderate positive relationship between 'new_vaccinations' and 'new_cases'. This suggests that in Germany, there is a tendency for the number of new cases to increase as the number of new vaccinations increases. However, this relationship is moderate and may be influenced by other factors.